# Portuguese → English Translation of Clinical Notes

This notebook applies three translation strategies to Portuguese obstetric
discharge notes prior to ICD-10 coding:

1. Google Translate
2. GPT-based translation
3. Sabiá-based translation

Translations are evaluated indirectly via downstream ICD-10 coding performance.
Clinical data are not distributed with this repository.

#Translation

In [ ]:
# Instalar a versão correta da biblioteca
!pip install googletrans==4.0.0-rc1

# Importações
from googletrans import Translator
import pandas as pd

In [ ]:
df = pd.read_pickle('/content/df.pkl')
df

In [ ]:
import pandas as pd
# Carrega o DataFrame
df = pd.read_pickle('/content/df.pkl')

# Inicializa o tradutor
translator = Translator()

# Traduz linha por linha da coluna '25. Evolução Alta'
def traduzir(texto):
    try:
        return translator.translate(texto, src='pt', dest='en').text
    except Exception as e:
        return f"[ERRO] {e}"

# Aplica a tradução e cria nova coluna
df['25. High Evolution'] = df['25. Evolução Alta'].astype(str).apply(traduzir)

# Visualiza as primeiras linhas
df[['25. Evolução Alta', '25. High Evolution']].head()

###Usando o GPT-4 Mini para Traduzir

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/df_traduzido.pkl')

In [ ]:
# Instalar/atualizar a biblioteca OpenAI e garantir httpx compatível
!pip install -U openai httpx --quiet

import pandas as pd
import time
from openai import OpenAI
from google.colab import files

# Inicializa o cliente OpenAI com sua chave
client = OpenAI(api_key="APIKEY")

# Função para traduzir uma linha com GPT-4o
def traduzir_com_gpt(texto):
    try:
        if pd.isna(texto) or texto.strip() == "":
            return ""
        resposta = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a professional medical translator. "
                        "Translate the following clinical report from Portuguese to English. "
                        "Expand all medical abbreviations and acronyms into their full English meanings. "
                        "Preserve the formal and clinical tone of the report."
                    )
                },
                {"role": "user", "content": texto}
            ],
            temperature=0
        )
        return resposta.choices[0].message.content.strip()
    except Exception as e:
        return f"[ERRO] {e}"

# Aplica a função com pausa e imprime linha por linha
traduzidos = []
for i, linha in enumerate(df['25. Evolução Alta']):
    traducao = traduzir_com_gpt(str(linha))
    traduzidos.append(traducao)
    print(f"\n--- Linha {i+1}/{len(df)} ---")
    print(f"Original:\n{linha}\n")
    print(f"Tradução:\n{traducao}\n")
    time.sleep(1)  # respeita os limites da API

# Salva no DataFrame
df['25. High Evolution (GPT 4o)'] = traduzidos

# Salva e faz o download do .pkl
df.to_pickle('/content/df_traduzido_gpt.pkl')
files.download('/content/df_traduzido_gpt.pkl')

In [ ]:
df

##Usando o Sabia

In [ ]:

# Instalar/atualizar a biblioteca OpenAI e garantir httpx compatível
!pip install -U openai httpx --quiet

import pandas as pd
import time
from openai import OpenAI
from google.colab import files

client = openai.OpenAI(
    api_key="API KEY",
    base_url="https://chat.maritaca.ai/api",
)

# Função para traduzir uma linha com GPT-4o
def traduzir_com_gpt(texto):
    try:
        if pd.isna(texto) or texto.strip() == "":
            return ""
        resposta = client.chat.completions.create(
            model="sabia-3.1",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "You are a professional medical translator. "
                        "Translate the following clinical report from Portuguese to English. "
                        "Expand all medical abbreviations and acronyms into their full English meanings. "
                        "Preserve the formal and clinical tone of the report."
                    )
                },
                {"role": "user", "content": texto}
            ],
            temperature=0
        )
        return resposta.choices[0].message.content.strip()
    except Exception as e:
        return f"[ERRO] {e}"

# Aplica a função com pausa e imprime linha por linha
traduzidos = []
for i, linha in enumerate(df['25. Evolução Alta']):
    traducao = traduzir_com_gpt(str(linha))
    traduzidos.append(traducao)
    print(f"\n--- Linha {i+1}/{len(df)} ---")
    print(f"Original:\n{linha}\n")
    print(f"Tradução:\n{traducao}\n")
    time.sleep(1)  # respeita os limites da API

# Salva no DataFrame
df['25. High Evolution (Maritalk)'] = traduzidos

# Salva e faz o download do .pkl
df.to_pickle('/content/df_traduzido_gpt_maritalk.pkl')
files.download('/content/df_traduzido_gpt_maritalk.pkl')